In [2]:
import pandas as pd
import json
import re

# Load the dataset
df = pd.read_csv('../data/raw/explaination_toxic_conversation.csv')

def format_conversation_to_json(df):
    """
    Formats a DataFrame with conversation data into a list of JSON objects.

    Args:
      df: A pandas DataFrame with the columns 'person_couple', 'name1', 'name2', and 'conversation'.

    Returns:
      A JSON formatted string representing the list of conversation objects.
    """
    conversations_list = []
    for index, row in df.iterrows():
        dialogue = []
        conversation_text = row['conversation']
        name1 = row['name1']
        name2 = row['name2']
        pattern = re.compile(f'({re.escape(name1)}|{re.escape(name2)}):')
        
        parts = pattern.split(conversation_text)

        if len(parts) > 1:
            current_parts = parts[1:]
            for i in range(0, len(current_parts), 2):
                speaker = current_parts[i].strip()
                if i + 1 < len(current_parts):
                    text = current_parts[i+1].strip().strip('"').strip()
                    if text: 
                        dialogue.append({"speaker": speaker, "text": text})
        else:
            lines = re.split(r'\d+\.\s*', conversation_text)
            lines = [line.strip().strip('"') for line in lines if line.strip()]
            for i, line in enumerate(lines):
                speaker = name1 if i % 2 == 0 else name2
                line = re.sub(f'^({re.escape(name1)}|{re.escape(name2)}):\s*', '', line)
                if line: 
                    dialogue.append({"speaker": speaker, "text": line.strip()})

        conversation_obj = {
            "conversation_id": index, 
            "person_couple": row['person_couple'],
            "name1": name1,
            "name2": name2,
            "dialogue": dialogue
        }
        conversations_list.append(conversation_obj)
    
    return json.dumps(conversations_list, indent=4, ensure_ascii=False)

formatted_json = format_conversation_to_json(df)

with open('formatted_conversations.json', 'w', encoding='utf-8') as f:
    f.write(formatted_json)

print("Example of a formatted conversation:")
print(json.dumps(json.loads(formatted_json)[551], indent=4, ensure_ascii=False))


Example of a formatted conversation:
{
    "conversation_id": 551,
    "person_couple": "Psicopatico e Adulatrice",
    "name1": "Ferido",
    "name2": "Filippella",
    "dialogue": [
        {
            "speaker": "Ferido",
            "text": "Non mi aspetto che tu sia intelligente, ma potresti almeno essere onesta e riconoscere che la tua opinione è sbagliata."
        },
        {
            "speaker": "Filippella",
            "text": "Non so cosa dire, sei così intelligente e perspicace. Non riesco a immaginare come tu riesca a vedere tutto con tanta chiarezza."
        },
        {
            "speaker": "Ferido",
            "text": "Sembra che tu sia più interessata a essere amata che a essere intelligente. Non è meglio essere onesti con se stessi?"
        },
        {
            "speaker": "Filippella",
            "text": "Sì, non riesco a capire perché non riesco a vedere le cose come tu. È come se tu avessi una visione speciale che non riesco a comprendere."
        }